<a href="https://colab.research.google.com/github/PoojaB24/EcoAssistantAgent/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install flask flask-cors

This command installs the **Flask** and **Flask-CORS** packages, which are required to create a backend web API for the model. It enables building a server that can handle requests from different origins (like a browser or frontend application).


In [6]:
!pip install -q flask flask-cors pyngrok joblib scikit-learn

This command silently installs all the required backend and deployment libraries, including **Flask**, **Flask-CORS**, **pyngrok**, **Joblib**, and **scikit-learn**. These packages are needed to serve the trained model as an API and connect it to external applications.


In [2]:
# -------------------------
# Colab cell: Start API + ngrok (Cleaned & Fixed)
# -------------------------

import os, threading, time, re, json, joblib
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok, conf

# --- CONFIG ---
MODEL_PATH = "eco_friendly_model_optimized.joblib"
FLASK_PORT = 5001
NGROK_AUTHTOKEN = "35N7xgmd28BpiTM1J3tX0JKZFuT_7KbNCJan4kTbTbrU3ektX"

# --- CLEAN OLD PROCESSES ---
!fuser -k 5001/tcp || true
!killall ngrok || true

# --- SETUP NGROK AUTH ---
conf.get_default().auth_token = NGROK_AUTHTOKEN

# --- VERIFY MODEL EXISTS ---
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model not found: {MODEL_PATH}")

# --- LOAD PIPELINE MODEL ---
model = joblib.load(MODEL_PATH)
print("✅ Pipeline model loaded (TF-IDF + classifier).")

# --- TEXT CLEANER ---
def clean_text(s):
    s = str(s).lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# --- FLASK APP ---
app = Flask("eco_assistant_api")
CORS(app)

@app.route("/health", methods=["GET"])
def health():
    return jsonify({"status": "ok", "model_loaded": True})

@app.route("/predict", methods=["POST"])
def predict():
    payload = request.get_json(force=True, silent=True) or {}
    text = payload.get("text", "")
    if not text:
        return jsonify({"error": "no text provided"}), 400

    cleaned = clean_text(text)

    # ✅ Predict using pipeline
    pred = model.predict([cleaned])[0]

    # ✅ Convert numpy type to Python int
    if hasattr(pred, "item"):
        pred = pred.item()

    # ✅ Map numeric class to readable label
    label_map = {
        0: "Not Eco-Friendly",
        1: "Moderate",
        2: "Eco-Friendly"
    }
    label = label_map.get(pred, str(pred))  # fallback if unseen label

    # ✅ Confidence score
    try:
        proba = float(model.predict_proba([cleaned]).max())
    except AttributeError:
        proba = 1.0

    # Simple keyword-based reason
    reasons = []
    words = cleaned.split()
    if any(w in words for w in ["organic", "recycled", "biodegradable", "sustainable", "jute", "bamboo"]):
        reasons.append("contains positive sustainability indicators")
    if any(w in words for w in ["plastic", "polyester", "synthetic", "nylon", "chemical"]):
        reasons.append("contains non-eco materials")

    return jsonify({
        "label": label,
        "confidence": round(proba * 100, 2),
        "reasons": reasons
    })


# --- RUN FLASK IN THREAD ---
def run_flask():
    app.run(host="0.0.0.0", port=FLASK_PORT, debug=False)

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()
time.sleep(2)

# --- NGROK SETUP ---
public_tunnel = ngrok.connect(addr=FLASK_PORT, proto="http")
public_url = public_tunnel.public_url
print(f"✅ Flask running on port {FLASK_PORT}")
print(f"🌐 Public URL: {public_url}/predict")

print("\nExample test:")
print(f"curl -X POST -H 'Content-Type: application/json' -d '{{\"text\":\"organic cotton eco bag\"}}' {public_url}/predict")

ngrok: no process found
✅ Pipeline model loaded (TF-IDF + classifier).
 * Serving Flask app 'eco_assistant_api'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://172.28.0.12:5001
INFO:werkzeug:Press CTRL+C to quit


✅ Flask running on port 5001
🌐 Public URL: https://luteotropic-aglimmer-rachelle.ngrok-free.dev/predict

Example test:
curl -X POST -H 'Content-Type: application/json' -d '{"text":"organic cotton eco bag"}' https://luteotropic-aglimmer-rachelle.ngrok-free.dev/predict


This block sets up a **Flask API** in Google Colab, loads the trained eco-friendly model, and exposes it to the internet using **ngrok** for external access. It creates endpoints for health checking and prediction, cleans input text, returns the predicted label with confidence and reasons, and prints a public URL that can be used to send real-time requests from a frontend.


In [3]:
import requests
response = requests.post("https://luteotropic-aglimmer-rachelle.ngrok-free.dev/predict", json={"text": "organic cotton eco bag"})
print(response.status_code, response.text)

INFO:werkzeug:127.0.0.1 - - [21/Nov/2025 08:22:50] "POST /predict HTTP/1.1" 200 -


200 {"confidence":100.0,"label":"Eco-Friendly","reasons":["contains positive sustainability indicators"]}



This block sends a POST request to your deployed **ngrok prediction API** with a sample product description in JSON format. It then prints the response status code and the prediction result returned by the model, verifying that the API is working correctly.
